In [1]:
import pandas as pd
import sqlite3

In [2]:
# conectamos con la base de datos my_database.db
connection = sqlite3.connect("my_database.db")

# obtenemos un cursor que utilizamos para las queries
crsr = connection.cursor()


In [4]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [5]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

In [7]:
query = '''
CREATE TABLE IF NOT EXISTS reparto (
	CodE VARCHAR(2) NOT NULL,
	CodB VARCHAR(3) NOT NULL,
	CodC VARCHAR(2) NOT NULL,
	Fecha DATE NOT NULL,
	Cantidad SMALLINT,
	PRIMARY KEY (CodE,CodB,CodC,fecha)
);
'''
crsr.execute(query)

In [8]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

reparto


In [9]:
query = '''
CREATE TABLE IF NOT EXISTS cervezas (
    CodC VARCHAR(2),
    Envase VARCHAR(32),
    Capacidad FLOAT(2),
    Stock INT(5),
    PRIMARY KEY (CodC)
)
'''

crsr.execute(query)

In [10]:
query = '''
CREATE TABLE IF NOT EXISTS bares (
    CodB VARCHAR(2),
    Nombre VARCHAR(32),
    Cif VARCHAR(32),
    Localidad VARCHAR(32),
    PRIMARY KEY (CodB)
)
'''

crsr.execute(query)

In [11]:
query = '''
CREATE TABLE IF NOT EXISTS empleados (
    CodE VARCHAR(2),
    Nombre VARCHAR(32),
    Sueldo INT(5),
    PRIMARY KEY (CodE)
)
'''

crsr.execute(query)


In [12]:
query = '''
INSERT OR REPLACE INTO empleados VALUES ('1','Prudencio Caminero',120000),
('2', 'Vicente Merario', 110000),
('3', 'Valentin Siempre', 100000)
'''
# CRUD Create Register(insert) Update Delete
crsr.execute(query)

In [13]:
query = '''
INSERT INTO reparto (CodE, CodB, CodC, Fecha, Cantidad) 
VALUES  ('1','001','01','2005-10-21',240 ),
( '1','001','02','2005-10-21',48 ),
( '1','002','03','2005-10-22',60 ),
( '1','004','05','2005-10-22',4 ),
( '2','002','03','2005-10-22',48 ),
( '2','002','05','2005-10-23',2 ),
( '2','004','01','2005-10-23',480 ),
( '2','004','02','2005-10-24',72 ),
( '3','003','03','2005-10-24',48 ),
( '3','003','04','2005-10-25',20 )
;
'''
crsr.execute(query)

In [14]:
query = '''
INSERT OR REPLACE INTO bares VALUES ('001','Stop', '11111111X','Villa Botijo'),
('002', 'Las Vegas', '22222222Y', 'Villa Botijo'),
('003', 'Club Social', NULL, 'Las Ranas'),
('004','Otra Ronda','33333333Z','La Esponja')
'''
crsr.execute(query)

In [15]:
query = '''
INSERT OR IGNORE INTO Cervezas VALUES ('01','Botella',0.2,3600),
('02', 'Botella', 0.33, 1200),
('03', 'Lata', 0.33, 2400),
('04','Botella',1,288),
('05','Barril',60,30)
'''
crsr.execute(query)

In [16]:
query = '''
SELECT * FROM reparto 
'''
sql_query(query)

,CodE,CodB,CodC,Fecha,Cantidad
0,1,001,01,2005-10-21,240
1,1,001,02,2005-10-21,48
2,1,002,03,2005-10-22,60
3,1,004,05,2005-10-22,4
4,2,002,03,2005-10-22,48
5,2,002,05,2005-10-23,2
6,2,004,01,2005-10-23,480
7,2,004,02,2005-10-24,72
8,3,003,03,2005-10-24,48
9,3,003,04,2005-10-25,20


In [29]:
# 1 Obtener el nombre de los empleados que hayan repartido al bar Stop durante la semana 
# del 17 al 23 de octubre de 2005.

query = '''
SELECT DISTINCT empleados.Nombre
FROM empleados
INNER JOIN reparto
ON empleados.CodE = reparto.CodE
INNER JOIN bares
ON bares.CodB = reparto.CodB
WHERE bares.nombre = 'Stop' AND reparto.Fecha BETWEEN '17/10/2005' AND '23/10/2005'
'''

sql_query(query)


,Nombre
0,Prudencio Caminero


In [ ]:
#2
query = '''
SELECT DISTINCT
bares.Cif,
bares.nombre
FROM bares
INNER JOIN reparto 
ON bares.CodB = reparto.CodB 
INNER JOIN cervezas
ON reparto.CodC = cervezas.CodC
WHERE cervezas.envase = 'Botella' and cervezas.Capacidad < 1
ORDER BY bares.Localidad
'''
sql_query(query)

,Cif,Nombre
0,33333333Z,Otra Ronda
1,11111111X,Stop


In [31]:
#3
query = '''
SELECT
bares.nombre,
cervezas.envase,
cervezas.capacidad,
reparto.Fecha,
reparto.Cantidad
FROM reparto
INNER JOIN empleados
ON reparto.CodE = empleados.CodE
INNER JOIN bares
ON reparto.CodB = bares.CodB
INNER JOIN cervezas
ON cervezas.CodC = reparto.CodC
WHERE empleados.Nombre = 'Prudencio Caminero'
'''
sql_query(query)

,Nombre,Envase,Capacidad,Fecha,Cantidad
0,Stop,Botella,0.20,2005-10-21,240
1,Stop,Botella,0.33,2005-10-21,48
2,Las Vegas,Lata,0.33,2005-10-22,60
3,Otra Ronda,Barril,60.00,2005-10-22,4


In [32]:
#4
query = '''
SELECT b.nombre, c.envase, c.capacidad

FROM bares b
INNER JOIN reparto r
ON b.Codb = r.Codb
INNER JOIN cervezas c
ON r.Codc = c.Codc
WHERE c.envase = 'Botella' AND c.capacidad IN (0.2, 0.33)

'''
sql_query(query)

,Nombre,Envase,Capacidad
0,Stop,Botella,0.20
1,Stop,Botella,0.33
2,Otra Ronda,Botella,0.20
3,Otra Ronda,Botella,0.33


In [ ]:
#5
query = '''
SELECT e.nombre as nombreE, b.nombre as nombreB, c.envase

FROM bares b
INNER JOIN reparto r
ON b.Codb = r.Codb
INNER JOIN cervezas c
ON r.Codc = c.Codc
INNER JOIN empleadoS e
ON e.CodE = r.CodE
WHERE c.envase = 'Botella' AND b.nombre  IN ("Stop", "Las Vegas")

'''
sql_query(query)

In [ ]:
#5 
query = '''
SELECT e.nombre as nombreE, b.nombre as nombreB, c.envase

FROM bares b
INNER JOIN reparto r
ON b.Codb = r.Codb
INNER JOIN cervezas c
ON r.Codc = c.Codc
INNER JOIN empleadoS e
ON e.CodE = r.CodE

GROUP BY e.nombre 
HAVING c.envase = 'Botella'

In [37]:
#6
query = '''
SELECT 
e.nombre AS nombreE, b.nombre AS nombreB, b.localidad,  r.fecha
--e.nombre, COUNT(distinct r.fecha) AS viajes
FROM empleados e
INNER JOIN reparto r
ON e.CodE = r.CodE
INNER JOIN bares b
ON b.CodB = r.CodB
WHERE b.localidad != 'Villa Botijo'

--GROUP BY CodE
--ORDER BY COUNT( DISTINCT Fecha) DESC

'''
sql_query(query)

,nombreE,nombreB,Localidad,Fecha
0,Prudencio Caminero,Otra Ronda,La Esponja,2005-10-22
1,Vicente Merario,Otra Ronda,La Esponja,2005-10-23
2,Vicente Merario,Otra Ronda,La Esponja,2005-10-24
3,Valentin Siempre,Club Social,Las Ranas,2005-10-24
4,Valentin Siempre,Club Social,Las Ranas,2005-10-25


In [41]:
query = '''
SELECT 
-- e.nombre AS nombreE, b.nombre AS nombreB, b.localidad, count(distinct r.fecha)
e.nombre,  count(distinct r.fecha) AS viajes
FROM empleados e
INNER JOIN reparto r
ON e.CodE = r.CodE
INNER JOIN bares b
ON b.CodB = r.CodB

GROUP BY e.CodE, b.localidad
HAVING b.localidad != 'Villa Botijo'

'''
sql_query(query)

,Nombre,viajes
0,Prudencio Caminero,1
1,Vicente Merario,2
2,Valentin Siempre,2


In [39]:
query = '''
SELECT 
-- e.nombre AS nombreE, b.nombre AS nombreB, b.localidad, count(distinct r.fecha)
e.nombre,  count(distinct r.fecha) AS viajes
FROM empleados e
INNER JOIN reparto r
ON e.CodE = r.CodE
INNER JOIN bares b
ON b.CodB = r.CodB
WHERE b.localidad != 'Villa Botijo'
GROUP BY e.CodE

'''
sql_query(query)

,Nombre,viajes
0,Prudencio Caminero,1
1,Vicente Merario,2
2,Valentin Siempre,2


In [ ]:
query = '''
SELECT e.nombre as nombreE, c.envase

FROM bares b
INNER JOIN reparto r
ON b.Codb = r.Codb
INNER JOIN cervezas c
ON r.Codc = c.Codc
INNER JOIN empleadoS e
ON e.CodE = r.CodE
WHERE c.envase = 'Botella' AND b.nombre = "Las Vegas"

INTERSECT

SELECT e.nombre as nombreE, c.envase

FROM bares b
INNER JOIN reparto r
ON b.Codb = r.Codb
INNER JOIN cervezas c
ON r.Codc = c.Codc
INNER JOIN empleadoS e
ON e.CodE = r.CodE
WHERE c.envase = 'Botella' AND b.nombre = "Stop"

'''
sql_query(query)

In [42]:
# 7
query = '''
SELECT t.nombre, t.localidad, max(t.maximo) AS maximo
FROM
(SELECT 
b.nombre, b.localidad, max(r.cantidad) as maximo

FROM cervezas c
INNER JOIN reparto r
ON c.CodC = r.CodC
INNER JOIN bares b
ON b.CodB = r.CodB

GROUP BY b.nombre, b.localidad) t


'''
sql_query(query)


,nombre,localidad,maximo
0,Otra Ronda,La Esponja,480


In [43]:
#8
query = '''
SELECT 
DISTINCT b.nombre

FROM bares b
INNER JOIN reparto r
ON b.CodB= r.CodB
INNER JOIN cervezas c
ON c.CodC = r.CodC
WHERE c.envase = 'Botella' AND c.Capacidad <1

'''
sql_query(query)

,Nombre
0,Stop
1,Otra Ronda


In [ ]:
query9 = '''
SELECT t.Sueldo*1.05, MAX(t.maximo) ,t.Nombre
FROM (SELECT e.Sueldo, COUNT(DISTINCT r.Fecha) as maximo, e.Nombre
    FROM empleados as e
    INNER JOIN reparto as r
    ON e.CodE = r.CodE
    GROUP BY e.Nombre) t
'''
sql_query(query9)

In [ ]:
query = '''
UPDATE empleados
SET sueldo = sueldo * 1.05
WHERE CodE IN (
SELECT 
CodE
FROM reparto
GROUP BY CodE
ORDER BY COUNT( DISTINCT Fecha) DESC LIMIT 1)
'''
crsr.execute(query)

In [59]:
query = '''

SELECT 
CodE
FROM reparto
GROUP BY CodE
ORDER BY COUNT( DISTINCT Fecha) DESC LIMIT 1
'''
sql_query(query)

,CodE
0,2


In [ ]:
connection.commit()
connection.close()